In [1]:
import pandas as pd
from sqlalchemy import create_engine

### Extract CSVs into DataFrames

In [22]:
vvr_data_world_file = "/Users/alexanderlorin/code/RAM_project/Resources/vegetarian_restaurants_US_datafiniti_vegetarian_restaurants_US_datafiniti.csv"
vvr_df = pd.read_csv(vvr_data_world_file)
vvr_indexed_df = vvr_df.reset_index(drop=False, inplace=False, col_fill='')
vvr_indexed_df.head()

,index,address,categories,city,claimed,country,cuisines,dateOpened,dateUpdated,descriptions,...,menuURL,name,paymentTypes,phones,postalCode,priceRange,province,sic,twitter,websites
0,0,2871 Gulf To Bay Blvd,"American, American (new), Clearwater Restauran...",Clearwater,NaN,US,NaN,NaN,2014-02-01T04:41:06Z,"Tags: American (New), American, and Trendy Sta...",...,NaN,Village Inn Restaurant and Bakery,NaN,7277964988,33759,NaN,FL,8423,NaN,NaN
1,1,605 W Main St,Caf and Sandwich Place,Louisville,NaN,US,"Vegetarian, Delicatessen",NaN,2016-07-23T22:37:24Z,"Atlantic No. 5, Louisville: See 4 unbiased rev...",...,NaN,Atlantic No. 5,Yes,5028833398,40202,USD 25.00-40.00,KY,NaN,NaN,atlanticno5.com
2,2,3803 9th St S W,"American, Barbecue, Puyallup Restaurants, Barb...",Puyallup,NaN,US,NaN,NaN,2015-11-19T22:22:31Z,"Description: Famous Dave's is a Barbecue , Am...",...,NaN,Famous Dave's,NaN,2536040340,98373,USD 10.00 - USD 15.00,WA,5610,NaN,http://famousdaves.com
3,3,30 S Main St,"Coffee & Tea, Restaurant, Restaurants, Bar, Am...",Oberlin,NaN,US,"Coffee & Tea, Restaurants, American, Bar, Pu...",NaN,2016-06-20T17:47:10Z,"The Feve, Oberlin: See 170 unbiased reviews of...",...,/oberlin-oh/mip/the-feve-8940755/menu,The Feve,"mastercard, amex, discover, Yes, visa","[""4407741978"",""4197741978"",""4407753064""]",44074,USD 25.00-40.00,OH,NaN,NaN,http://www.thefeve.com
4,4,55 Crescent Dr,Sandwich Place and Fast Food Restaurant,Pleasant Hill,yellowpages.com,US,"Take Out Restaurants, Asian Restaurants, Resta...",NaN,2016-07-23T23:55:05Z,"[""https://www.zagat.com/r/yalla-mediterranean-...",...,/pleasant-hill-ca/mip/yalla-mediterranean-5061...,Quiznos,"master card, visa",9258258644,94523,$,CA,NaN,quiznos,http://www.yallamedi.com/


In [3]:
#use for second datatset

# county_file = "../Resources/CountyLicenseCount.csv"
# county_df = pd.read_csv(county_file)
# county_df.head()

### Transform premise DataFrame

In [36]:
# Create a filtered dataframe from specific columns
vvr_cols = ["index","address", "categories", "city", "country", "name", "phones", "postalCode", "priceRange", "province", "twitter", "websites", "facebookPageURL"]
vvr_transformed= vvr_indexed_df[vvr_cols].copy()

# Rename the column headers
vvr_transformed = vvr_transformed.rename(columns={"postalCode": "postalcode",
                                                  "priceRange": "pricerange",
                                                  "index":"id",
                                                  "facebookPageURL": "facebookpageurl"})

# Clean the data by dropping duplicates and setting the index
# vvr_transformed.drop_duplicates("id", inplace=True)
# vvr_transformed.set_index("id", inplace=True)

vvr_transformed.head()

,id,address,categories,city,country,name,phones,postalcode,pricerange,province,twitter,websites,facebookpageurl
0,0,2871 Gulf To Bay Blvd,"American, American (new), Clearwater Restauran...",Clearwater,US,Village Inn Restaurant and Bakery,7277964988,33759,NaN,FL,NaN,NaN,NaN
1,1,605 W Main St,Caf and Sandwich Place,Louisville,US,Atlantic No. 5,5028833398,40202,USD 25.00-40.00,KY,NaN,atlanticno5.com,NaN
2,2,3803 9th St S W,"American, Barbecue, Puyallup Restaurants, Barb...",Puyallup,US,Famous Dave's,2536040340,98373,USD 10.00 - USD 15.00,WA,NaN,http://famousdaves.com,NaN
3,3,30 S Main St,"Coffee & Tea, Restaurant, Restaurants, Bar, Am...",Oberlin,US,The Feve,"[""4407741978"",""4197741978"",""4407753064""]",44074,USD 25.00-40.00,OH,NaN,http://www.thefeve.com,NaN
4,4,55 Crescent Dr,Sandwich Place and Fast Food Restaurant,Pleasant Hill,US,Quiznos,9258258644,94523,$,CA,quiznos,http://www.yallamedi.com/,NaN


### Transform county DataFrame

In [32]:
# county_cols = ["ID", "County Name (Licensee)", "County ID Code", "License Count"]
# county_transformed = county_df[county_cols].copy()

# # Rename the column headers
# county_transformed = county_transformed.rename(columns={"ID": "id",
#                                                          "County Name (Licensee)": "county_name",
#                                                          "License Count": "license_count",
#                                                          "County ID Code": "county_id"})

# # Set index
# county_transformed.set_index("id", inplace=True)

# county_transformed.head()

### Create database connection

In [33]:
connection_string = "postgres:postgres@localhost:5432/vegan_ram"
engine = create_engine(f'postgresql://{connection_string}')

In [34]:
# Confirm tables
engine.table_names()

<ipython-input-34-2d97e9123e98>:2: SADeprecationWarning: The Engine.table_names() method is deprecated and will be removed in a future release.  Please refer to Inspector.get_table_names(). (deprecated since: 1.4)
  engine.table_names()


['vvr']

### Load DataFrames into database

In [35]:
vvr_transformed.to_sql(name='vvr', con=engine, if_exists='append', index=True)

DataError: (psycopg2.errors.InvalidTextRepresentation) invalid input syntax for type integer: "["77098"]"
LINE 1: ...'Houston', 'US', 'Hobbit Cafe', '["7135265460"]', '["77098"]...
                                                             ^

[SQL: INSERT INTO vvr (index, id, address, categories, city, country, name, phones, postalcode, pricerange, province, twitter, websites, facebookpageurl) VALUES (%(index)s, %(id)s, %(address)s, %(categories)s, %(city)s, %(country)s, %(name)s, %(phones)s, %(postalcode)s, %(pricerange)s, %(province)s, %(twitter)s, %(websites)s, %(facebookpageurl)s)]
[parameters: ({'index': 0, 'id': 0, 'address': '2871 Gulf To Bay Blvd', 'categories': 'American, American (new), Clearwater Restaurants, American (new) / American / Vegan', 'city': 'Clearwater', 'country': 'US', 'name': 'Village Inn Restaurant and Bakery', 'phones': '7277964988', 'postalcode': '33759', 'pricerange': None, 'province': 'FL', 'twitter': None, 'websites': None, 'facebookpageurl': None}, {'

In [ ]:
# county_transformed.to_sql(name='county', con=engine, if_exists='append', index=True)